In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
SIGMA_SL = -1
SIGMA_TP = 1.5
MAX_TRADE_DURATION = 63
PTF_VOL_TGT = 0.15
AUM = 100000
AUM_LOST_SL = 0.02

In [3]:
df = pd.read_csv('dataset_tmc.csv', parse_dates = ['Dates'], date_format = '%d/%m/%y')
df.columns = [i.lower() for i in df.columns]
df.columns = [i.replace(' ','_') for i in df.columns]
df = df.set_index('dates')

# Check NaNs
if df.isna().sum().sum() != 0:
    print('check NaNs in data')

# refill Sat&Sun
refilled = df.reindex(pd.date_range(df.index.min(), df.index.max()), method='ffill')

In [26]:
# hardcoded start allowing 5Y of data for Stdev calc and Max Duration of trades
startdates = list(df.loc['2018-03-12':df.tail(MAX_TRADE_DURATION).index.astype(str)[0]].index.astype(str))

trades_list = []

for i in range(100):
    security_id = random.choice(list(df.columns))
    trade_entry_date = random.choice(startdates)
    direction = random.choice([-1, 1])

    # compute 5Y Monthly standard deviation starting from date of trade entry and going backward
    hist_stdev = refilled.loc[refilled.index.isin(pd.date_range(end=trade_entry_date, periods=60+1, freq=pd.DateOffset(months=1))), security_id
                    ].pct_change().dropna().std() #* np.sqrt(12)

    trade_pnl = df.loc[trade_entry_date:, [security_id]].head(MAX_TRADE_DURATION+1)
    trade_pnl['security_id'] = security_id
    trade_pnl['direction'] = direction
    trade_pnl['hist_volatility'] = hist_stdev
    trade_pnl['entry_price'] = df.loc[trade_entry_date, security_id]
    trade_pnl['tp_price'] = trade_pnl['entry_price'] * (1 + hist_stdev * SIGMA_TP * direction)
    trade_pnl['sl_price'] = trade_pnl['entry_price'] * (1 + hist_stdev * SIGMA_SL * direction)
    trade_pnl['quantity'] = (AUM * AUM_LOST_SL) / ((trade_pnl['entry_price'] - trade_pnl['sl_price']) * direction)

    # check if TP/SL have been hitted 
    if direction == 1:
        trade_pnl['tp_hit'] = (trade_pnl[security_id] > trade_pnl['tp_price']) * 1
        trade_pnl['sl_hit'] = (trade_pnl[security_id] < trade_pnl['sl_price']) * 1
    elif direction == -1:
        trade_pnl['tp_hit'] = (trade_pnl[security_id] < trade_pnl['tp_price']) * 1
        trade_pnl['sl_hit'] = (trade_pnl[security_id] > trade_pnl['sl_price']) * 1

    trade_pnl['tp_sl_hit'] = trade_pnl['tp_hit'] + trade_pnl['sl_hit']

    # make exit condition from the trade explicit
    if trade_pnl['tp_sl_hit'].sum()==0:
        trade_pnl['exit_condition'] = 'max_duration'
        trade_pnl['exit_price'] = trade_pnl[security_id].iloc[-1]
    else:
        trade_pnl = trade_pnl.loc[:trade_pnl[trade_pnl['tp_sl_hit'] == 1].index[0]].copy()
        trade_pnl['exit_condition'] = 'stop_loss' if trade_pnl['sl_hit'].sum()==1 else 'take_profit'
        trade_pnl['exit_price'] = trade_pnl['sl_price'] if trade_pnl['sl_hit'].sum()==1 else trade_pnl['tp_price']

    # first day must be subtracted as it is the day when the trade is open
    trade_pnl['duration'] = len(trade_pnl)-1

    # store the trade
    trades_list.append(trade_pnl[['security_id', 'direction', 'hist_volatility', 'entry_price',
       'tp_price', 'sl_price', 'quantity', 'exit_condition', 'exit_price', 'duration']].drop_duplicates())

In [27]:
pd.concat(trades_list)

,security_id,direction,hist_volatility,entry_price,tp_price,sl_price,quantity,exit_condition,exit_price,duration
dates,,,,,,,,,,
2018-05-23,us_2y,1,0.002057,83.240,83.496856,83.068763,11679.716824,take_profit,83.496856,4
2021-06-16,silver,1,0.085005,25.260,28.480851,23.112766,931.430878,stop_loss,23.112766,29
2019-07-01,us_2y,1,0.002794,84.570,84.924461,84.333693,8463.562471,take_profit,84.924461,33
2022-06-28,nasdaq,1,0.059358,283.540,308.785381,266.709746,118.833618,take_profit,308.785381,22
2019-02-04,uk_15y,1,0.029095,64.745,67.570681,62.861213,1061.690974,take_profit,67.570681,33
...,...,...,...,...,...,...,...,...,...,...
2018-10-12,germany_10y,1,0.010782,138.770,141.014269,137.273821,1336.738293,take_profit,141.014269,39
2018-12-13,us_staples,1,0.029728,55.440,57.912182,53.791879,1213.502799,stop_loss,53.791879,2
2022-03-07,eu_healthcare,-1,0.036192,97.540,92.244678,101.070214,566.537836,stop_loss,101.070214,6
